INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [24]:
# Library imports
import numpy as np

Create classes for modularity

In [25]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [26]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [27]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [28]:
# Start of Optimizers

class Optimizer_SGD:
    """SGD optimizer extended with learning-rate decay, momentum, and Adagrad support.

    Parameters
    ----------
    learning_rate : float
        Initial learning rate.
    decay : float
        Learning rate decay per iteration (default 0 = no decay). Uses: lr = initial_lr / (1 + decay * iteration).
    momentum : float
        Momentum factor (0 = disabled).
    use_adagrad : bool
        If True, use Adagrad adaptive updates instead of plain SGD/momentum.
    epsilon : float
        Small value to avoid division by zero in Adagrad.
    """
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, use_adagrad=False, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.use_adagrad = use_adagrad
        self.epsilon = epsilon

    def pre_update_params(self):
        """Call at the start of an update step (e.g., each epoch) to adjust the learning rate if decay is used."""
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))
        else:
            self.current_learning_rate = self.learning_rate

    # Update the parameters of a layer
    def update_params(self, layer):
        # If using momentum, ensure the layer has momentum arrays
        if self.momentum and not self.use_adagrad:
            if not hasattr(layer, 'weight_momentum'):
                layer.weight_momentum = np.zeros_like(layer.weights)
                layer.bias_momentum = np.zeros_like(layer.biases)

            # Build weight and bias updates
            weight_updates = self.momentum * layer.weight_momentum - self.current_learning_rate * layer.dweights
            bias_updates = self.momentum * layer.bias_momentum - self.current_learning_rate * layer.dbiases

            # Store momentum
            layer.weight_momentum = weight_updates
            layer.bias_momentum = bias_updates

            # Apply updates
            layer.weights += weight_updates
            layer.biases += bias_updates

        # If using Adagrad
        elif self.use_adagrad:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)

            # Accumulate squared gradients
            layer.weight_cache += layer.dweights ** 2
            layer.bias_cache += layer.dbiases ** 2

            # Update parameters using Adagrad rule
            layer.weights += - self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
            layer.biases += - self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

        # Plain SGD (no momentum, no adaptive)
        else:
            layer.weights += - self.current_learning_rate * layer.dweights
            layer.biases += - self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        # Increment iteration count after updates
        self.iterations += 1

In [29]:
# Optimizer 1: SGD with optional learning-rate decay
class Optimizer_SGD_Decay:
    def __init__(self, learning_rate=1.0, decay=0.0):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))
        else:
            self.current_learning_rate = self.learning_rate

    def update_params(self, layer):
        layer.weights += - self.current_learning_rate * layer.dweights
        layer.biases += - self.current_learning_rate * layer.dbiases

    def post_update_params(self):
        self.iterations += 1


# Optimizer 2: SGD with Momentum
class Optimizer_SGD_Momentum:
    def __init__(self, learning_rate=1.0, momentum=0.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.momentum = momentum
        self.iterations = 0

    def pre_update_params(self):
        # No decay in this variant; keep current_learning_rate equal to learning_rate
        self.current_learning_rate = self.learning_rate

    def update_params(self, layer):
        if not hasattr(layer, 'weight_momentum'):
            layer.weight_momentum = np.zeros_like(layer.weights)
            layer.bias_momentum = np.zeros_like(layer.biases)

        # Compute updates
        weight_updates = self.momentum * layer.weight_momentum - self.current_learning_rate * layer.dweights
        bias_updates = self.momentum * layer.bias_momentum - self.current_learning_rate * layer.dbiases

        # Store momentum
        layer.weight_momentum = weight_updates
        layer.bias_momentum = bias_updates

        # Apply updates
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


# Optimizer 3: Adagrad (adaptive gradient)
class Optimizer_Adagrad:
    def __init__(self, learning_rate=1.0, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.epsilon = epsilon
        self.iterations = 0

    def pre_update_params(self):
        self.current_learning_rate = self.learning_rate

    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Accumulate squared gradients
        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2

        # Update parameters
        layer.weights += - self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += - self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    def post_update_params(self):
        self.iterations += 1

Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [30]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [13]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [31]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Choose an optimizer by uncommenting one of the following lines or instantiate your desired optimizer
# Example: SGD with decay
# optimizer = Optimizer_SGD_Decay(learning_rate=0.1, decay=1e-3)
# Example: SGD with momentum
# optimizer = Optimizer_SGD_Momentum(learning_rate=0.1, momentum=0.9)
# Example: Adagrad
# optimizer = Optimizer_Adagrad(learning_rate=0.1, epsilon=1e-7)

# Default: use SGD with small learning rate and slight decay
optimizer = Optimizer_SGD_Decay(learning_rate=0.1, decay=1e-4)

PERFORM ONLY 1 PASS

In [23]:
# Training loop: forward pass, backpropagation, and weight update for 1000 epochs
epochs = 1000
for epoch in range(1, epochs + 1):
    # Allow optimizer to adjust learning rate or perform setup
    optimizer.pre_update_params()

    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    # Loss and accuracy
    loss = loss_function.calculate(activation2.output, y)
    predictions = np.argmax(activation2.output, axis=1)
    if len(y.shape) == 2:
        y_true = np.argmax(y, axis=1)
    else:
        y_true = y
    accuracy = np.mean(predictions == y_true)

    # Backward pass (from loss to inputs)
    loss_function.backward(activation2.output, y)
    dvalues = loss_function.dinputs

    activation2.backward(dvalues)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

    # Print progress every 100 epochs (and always on the final epoch)
    if epoch % 100 == 0 or epoch == epochs:
        print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}, lr: {optimizer.current_learning_rate}")

epoch: 100, acc: 0.377, loss: 1.099, lr: 0.09901970492127933
epoch: 200, acc: 0.397, loss: 1.099, lr: 0.09804882831650162
epoch: 200, acc: 0.397, loss: 1.099, lr: 0.09804882831650162
epoch: 300, acc: 0.383, loss: 1.099, lr: 0.09709680551509856
epoch: 300, acc: 0.383, loss: 1.099, lr: 0.09709680551509856
epoch: 400, acc: 0.383, loss: 1.099, lr: 0.09616309260505818
epoch: 400, acc: 0.383, loss: 1.099, lr: 0.09616309260505818
epoch: 500, acc: 0.387, loss: 1.099, lr: 0.0952471663967997
epoch: 500, acc: 0.387, loss: 1.099, lr: 0.0952471663967997
epoch: 600, acc: 0.387, loss: 1.098, lr: 0.09434852344560807
epoch: 600, acc: 0.387, loss: 1.098, lr: 0.09434852344560807
epoch: 700, acc: 0.387, loss: 1.098, lr: 0.09346667912889055
epoch: 700, acc: 0.387, loss: 1.098, lr: 0.09346667912889055
epoch: 800, acc: 0.387, loss: 1.098, lr: 0.09260116677470136
epoch: 800, acc: 0.387, loss: 1.098, lr: 0.09260116677470136
epoch: 900, acc: 0.387, loss: 1.098, lr: 0.09175153683824204
epoch: 900, acc: 0.387, lo

# Optimizer comparison: stabilization and accuracy

This section trains the same model twice with two different optimizers and reports:
- The first epoch at which loss stabilizes (plateaus within a small tolerance across a window)
- The final accuracy after all epochs

You can change which two optimizers to compare and their hyperparameters in the code cell below.

In [32]:
import copy

# Helper: train the model with a given optimizer factory and return history
# Note: we clone layers to avoid reusing trained weights across runs.
def train_with_optimizer(optimizer_factory, epochs=1000, log_every=0):
    # Fresh layers for each run
    l1 = Layer_Dense(2, 3)
    act1 = Activation_ReLU()
    l2 = Layer_Dense(3, 3)
    act2 = Activation_Softmax()
    loss_fn = Loss_CategoricalCrossEntropy()
    opt = optimizer_factory()

    history_loss = []
    history_acc = []

    for epoch in range(1, epochs + 1):
        # Pre-update (for lr schedules)
        opt.pre_update_params()

        # Forward
        l1.forward(X)
        act1.forward(l1.output)
        l2.forward(act1.output)
        act2.forward(l2.output)

        # Loss / acc
        loss = loss_fn.calculate(act2.output, y)
        preds = np.argmax(act2.output, axis=1)
        y_true_local = np.argmax(y, axis=1) if len(y.shape) == 2 else y
        acc = np.mean(preds == y_true_local)

        history_loss.append(loss)
        history_acc.append(acc)

        # Backward
        loss_fn.backward(act2.output, y)
        act2.backward(loss_fn.dinputs)
        l2.backward(act2.dinputs)
        act1.backward(l2.dinputs)
        l1.backward(act1.dinputs)

        # Update
        opt.update_params(l1)
        opt.update_params(l2)
        opt.post_update_params()

        if log_every and (epoch % log_every == 0 or epoch == epochs):
            print(f"[run] epoch {epoch} | loss {loss:.4f} | acc {acc:.3f}")

    return {
        'loss': np.array(history_loss),
        'acc': np.array(history_acc)
    }


def stabilization_epoch(losses, window=20, tol=1e-3):
    """Return the first epoch index (1-based) where loss change in the trailing
    window is within tol (i.e., stabilized). If never stabilizes, return None."""
    if len(losses) < window:
        return None
    for i in range(window, len(losses) + 1):
        seg = losses[i - window:i]
        if seg.max() - seg.min() <= tol:
            return i
    return None

# Choose two optimizers to compare by providing factories (callables returning new instances)
optA_factory = lambda: Optimizer_SGD_Decay(learning_rate=0.1, decay=1e-4)
optB_factory = lambda: Optimizer_Adagrad(learning_rate=0.1, epsilon=1e-7)

# Train both
print("Training with Optimizer A (SGD + decay)...")
resA = train_with_optimizer(optA_factory, epochs=1000, log_every=0)
print("Training with Optimizer B (Adagrad)...")
resB = train_with_optimizer(optB_factory, epochs=1000, log_every=0)

# Compute stabilization and final metrics
stabA = stabilization_epoch(resA['loss'], window=20, tol=1e-3)
stabB = stabilization_epoch(resB['loss'], window=20, tol=1e-3)

final_accA = float(resA['acc'][-1])
final_accB = float(resB['acc'][-1])
final_lossA = float(resA['loss'][-1])
final_lossB = float(resB['loss'][-1])

print("\nComparison Summary:")
print(f"- Optimizer A (SGD+decay): stabilized at epoch {stabA if stabA else 'N/A'}, final loss {final_lossA:.4f}, final acc {final_accA:.3f}")
print(f"- Optimizer B (Adagrad):   stabilized at epoch {stabB if stabB else 'N/A'}, final loss {final_lossB:.4f}, final acc {final_accB:.3f}")

Training with Optimizer A (SGD + decay)...
Training with Optimizer B (Adagrad)...

Comparison Summary:
- Optimizer A (SGD+decay): stabilized at epoch 20, final loss 1.0982, final acc 0.380
- Optimizer B (Adagrad):   stabilized at epoch 62, final loss 1.0844, final acc 0.383
